# Imports

In [5]:
import torch
import seaborn as sns
import matplotlib.pyplot as plt
import os
from sklearn.decomposition import PCA
from pprint import pprint
import pandas as pd

In [6]:
%load_ext autoreload
%autoreload 1
%aimport models
%aimport main
%aimport data_utils
%aimport contrastive_data
%aimport scoring

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
from contrastive_data import *
from data_utils import *
from main import *
from models import *
from scoring import *
from glob import glob

# Data

## Constants

(To change for villarica : use wrangled Nabid data and filter genes)

In [8]:
N = 70
data_path = '~/data/nabid/pilot2filtered.csv'


In [11]:
data = pd.read_csv(data_path)
genes = pd.read_csv(f'out/{N}_best_genes.csv', index_col=0)

In [ ]:
X = data[genes.index]

Unexpected exception formatting exception. Falling back to standard exception


## Run from here to re-split without reloading all data

(will be necessary if wtlike is changed however)

In [ ]:
#CELL TO CHANGE
run_name = 'cls_tp53_0_best'
run_dir = 'TP53_all_single/best'
BAG_SIZE = 50
dir_name = join('models',run_dir)
dataset_class = BatchDataset
! cat models/{run_dir}/config.ini

config-file = models/cls_tp53_0/cls_tp53_0_22/config.ini
dest-name = cls_tp53_0
verbose = 1
loss = standard
margin = 2.4360665047229384
dropout = 0.3565534119325581
weight-decay = 0.008166142039759013
batch-size = 128
positive-fraction = 0.472911131385171
shape = [718, 718, 718, 718]
embed-dim = 20
lr = 3.325281264031996e-05
scheduler = plateau
patience = 134
cosine-t = 207
task = classifier
alpha = 0
n-epochs = 600
knn = 5
n-workers = 0
bag-size = 0
mil-mode = attention


In [ ]:
df_train, df_seen, df_unseen = load_split(join(dir_name, 'split'), counts, )
df_dict = {
    'train':df_train,
    'seen':df_seen,
    'unseen':df_unseen,
}

47834, 15945, 19999 exemples in data
75 variants in train
24 variants in unseen


In [ ]:
n_mut = len([v for v in df_train['variant'].unique() if v in MAIN_MUTANTS])
print(f"Number of top {len(MAIN_MUTANTS)} variants in training set: {n_mut}")

Number of top 11 variants in training set: 8


In [ ]:
# obsolete - all runs should be grouped by control now
# print(run_name)
# group_original = True
# if ('control' == df_train.variant).any():
#     if  ('control' == df_unseen.variant).any():
#         print('WT-like not merged together in original data')
#         group_original = False
#     else:
#         print('WT-like merged together in original data, put in train/seen')
# else:
#     print('WT-like merged together in original data, put in unseen')

# print(f"{group_wt_like=}")

# if group_original ^ group_wt_like: # xor
#     raise ValueError(f'The data used to trained the model was {"not " if not group_original else ""}grouped \
#                      and the data now is {"not " if not group_wt_like else ""}grouped') 


In [ ]:
dl_train, dl_seen, dl_unseen = make_loaders(
    df_train, df_seen,df_unseen, batch_size=64, dataset_class=dataset_class,n_workers=1,pos_frac=0.5, 
    dataset_kwargs={'bag_size':BAG_SIZE})
dl_dict = {
    'train':dl_train,
    'seen':dl_seen,
    'unseen':dl_unseen,
}

In [ ]:
# model = torch.load(join(dir_name,'model.pkl'), map_location='cpu')
model :Model = torch.load(join(dir_name,'best_model.pkl'), map_location='cpu')
model.eval()

Classifier(
  (network): MLP(
    (layers): Sequential(
      (0): Linear(in_features=1224, out_features=718, bias=True)
      (1): ELU(alpha=1.0)
      (2): Dropout(p=0.3565534119325581, inplace=False)
      (3): Linear(in_features=718, out_features=718, bias=True)
      (4): ELU(alpha=1.0)
      (5): Dropout(p=0.3565534119325581, inplace=False)
      (6): Linear(in_features=718, out_features=718, bias=True)
      (7): ELU(alpha=1.0)
      (8): Dropout(p=0.3565534119325581, inplace=False)
      (9): Linear(in_features=718, out_features=718, bias=True)
      (10): ELU(alpha=1.0)
      (11): Dropout(p=0.3565534119325581, inplace=False)
      (12): Linear(in_features=718, out_features=20, bias=True)
    )
  )
  (output_layer): Linear(in_features=20, out_features=75, bias=True)
)

In [ ]:
!cat models/{run_dir}/best_score.json

{
  "5_nn_ref_seen": 0.029413609281906554,
  "i": 233
}

In [ ]:
fclass = variant_data.loc[df_seen.variant.unique(), 'Variant functional class']
fclass = fclass.where(fclass.index.map(lambda x :( x[0] != x[-1]) and (x!= 'WT')), 'Synonymous').sort_values(key=lambda s: s.map(VFC_KEY))